# Part 2 Feature Engineering

## 2.5 Repeat behavior

In [1]:
import pandas as pd
import numpy as np

In [2]:
user_data = pd.read_csv('cleaned_raw/user_data.csv')
user_data.head()

,Unnamed: 0,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,0,328862,323294,833,2882,2661.0,829,0
1,1,328862,844400,1271,2882,2661.0,829,0
2,2,328862,575153,1271,2882,2661.0,829,0
3,3,328862,996875,1271,2882,2661.0,829,0
4,4,328862,1086186,1271,1253,1049.0,829,0


### 2.5.1 For merchants: how many repeat buyers? What is the ratio of repeat buyer vs unique buyer for a certain merchant?  These are the features reflect the level of attractiveness 

In [48]:
repeat_purchase = user_data[(user_data['action_type'] == 2) & (user_data['time_stamp'] < 1111)]
repeat_purchase = repeat_purchase.groupby(['seller_id'])['user_id'].value_counts().to_frame()
repeat_purchase.columns = ['purchase_count']
repeat_purchase.reset_index(inplace=True)
repeat_purchase.head()

,seller_id,user_id,purchase_count
0,1,406,21
1,1,56832,21
2,1,180072,20
3,1,339584,15
4,1,88181,14


In [49]:
repeat_purchase_True = repeat_purchase[repeat_purchase['purchase_count'] > 1]
repeat_buyer_label = repeat_purchase_True[['user_id']].drop_duplicates()
repeat_count_each_seller = repeat_purchase_True.groupby(['seller_id'])['user_id'].nunique().reset_index()
repeat_count_each_seller.columns = ['seller_id', 'repeat_user_num']
all_seller_id = pd.DataFrame({'seller_id':range(1, 4997 ,1)})
repeat_count_each_seller = pd.merge(all_seller_id, repeat_count_each_seller, how='left',on=['seller_id']).fillna(0)

**this could reflect the attraction to repeat buyers**

In [50]:
repeat_count_each_seller.head()

,seller_id,repeat_user_num
0,1,2214.0
1,2,4.0
2,3,2.0
3,4,48.0
4,5,14.0


In [52]:
all_unique_user = repeat_purchase.groupby(['seller_id'])['user_id'].count().reset_index()
repeat_m = pd.merge(repeat_count_each_seller, all_unique_user, on='seller_id' )
repeat_m = repeat_m[['seller_id','repeat_user_num','user_id']]
repeat_m = repeat_m.rename(columns={'user_id':'all_unique_user'})

In [56]:
repeat_m.dtypes

seller_id            int64
repeat_user_num    float64
all_unique_user      int64
dtype: object

In [57]:
repeat_m['att_ratio'] = repeat_m['repeat_user_num'] / repeat_m['all_unique_user'].astype(float)

In [167]:
repeat_m.shape

(4990, 4)

In [166]:
repeat_m.to_csv('data_processed/repeat_merchant_result.csv')

### 2.5.1 For users: in how many stores did they do repeat buying? What is the average repeat buying action interval?

In [71]:
repeat_user = user_data[(user_data['action_type'] == 2) & (user_data['time_stamp'] < 1111)]

In [72]:
repeat_user.head()

,Unnamed: 0,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
19,19,328862,81766,614,4605,7622.0,709,2
23,23,328862,853901,267,3073,8126.0,805,2
24,24,328862,1102596,267,3073,8126.0,805,2
25,25,328862,963870,267,3073,8126.0,805,2
26,26,328862,813647,267,3073,8126.0,805,2


In [73]:
repeat_user = repeat_user.sort_values(['user_id','seller_id','time_stamp']).reset_index(drop=True)
repeat_user.head()

,Unnamed: 0,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,23288888,1,504149,1023,925,7402.0,1011,2
1,23288867,1,112203,1252,4026,1469.0,1021,2
2,26051110,2,644638,602,420,4953.0,626,2
3,26051111,2,1072591,1213,420,4953.0,626,2
4,26051112,2,239288,602,420,4953.0,626,2


In [75]:
repeat_buyer_ls = repeat_purchase_True['user_id'].unique()

In [76]:
repeat_buyer_ls

array([   406,  56832, 180072, ..., 387267, 414417, 421872])

In [117]:
repeat_user_t = repeat_user.loc[repeat_user['user_id'].isin(repeat_buyer_ls)]

In [118]:
repeat_user_t.shape

(1731024, 8)

In [119]:
repeat_user.shape

(2068790, 8)

In [120]:
repeat_user_t.head()

,Unnamed: 0,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
2,26051110,2,644638,602,420,4953.0,626,2
3,26051111,2,1072591,1213,420,4953.0,626,2
4,26051112,2,239288,602,420,4953.0,626,2
5,26051093,2,864941,737,1974,8321.0,817,2
6,26051137,2,1068315,703,2076,1283.0,721,2


In [121]:
#repeat_user_se = repeat_user_t.head(100)

In [122]:
repeat_user_t['diff'] = repeat_user_t.groupby(['user_id','seller_id'])['time_stamp'].diff()

/Users/xiaoguihua/anaconda3/envs/gabriel/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
repeat_user_t1 = repeat_user_t.dropna(subset=['diff'])
repeat_user_t1.shape

(709772, 9)

In [146]:
repeat_user_t1.head()

,Unnamed: 0,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,diff
3,26051111,2,1072591,1213,420,4953.0,626,2,0.0
4,26051112,2,239288,602,420,4953.0,626,2,0.0
8,26051128,2,917794,1401,3552,2276.0,1010,2,0.0
10,25403672,3,895754,1505,795,3608.0,516,2,0.0
13,37517291,5,841334,1439,360,6880.0,903,2,0.0


In [147]:
conditions =[
    (repeat_user_t1['diff'] < 35.0),
    (repeat_user_t1['diff'] >= 35.0) & (repeat_user_t1['diff'] < 135.0),
    (repeat_user_t1['diff'] >= 135.0) & (repeat_user_t1['diff'] < 235.0),
    (repeat_user_t1['diff'] >= 235.0) & (repeat_user_t1['diff'] < 335.0),
    (repeat_user_t1['diff'] >= 335.0) & (repeat_user_t1['diff'] < 435.0),
    (repeat_user_t1['diff'] >= 435.0) & (repeat_user_t1['diff'] < 535.0),
    (repeat_user_t1['diff'] >= 535.0) & (repeat_user_t1['diff'] < 635.0),
]

choices = [repeat_user_t1['diff'], repeat_user_t1['diff']-70,  repeat_user_t1['diff']-140,
           repeat_user_t1['diff']-210, repeat_user_t1['diff']-280, repeat_user_t1['diff']-350,
          repeat_user_t1['diff']-420]

repeat_user_t1['diff_new'] = np.select(conditions, choices, default=0)

/Users/xiaoguihua/anaconda3/envs/gabriel/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [160]:
repeat_user_result = repeat_user_t1[['user_id','seller_id', 'diff_new']].reset_index(drop=True)

In [161]:
repeat_user_result.head()

,user_id,seller_id,diff_new
0,2,420,0.0
1,2,420,0.0
2,2,3552,0.0
3,3,795,0.0
4,5,360,0.0


In [162]:
repeat_user_result = pd.DataFrame(repeat_user_result.groupby(['user_id', 'seller_id'])['diff_new'].mean().reset_index())

In [165]:
repeat_user_result.to_csv('data_processed/repeat_user_result.csv')